# Airbnb Boston Analysis

In [1]:
# Colocar um texto falando do Airbnb e, depois, apresentar os problemas como o CRISP-DM começando com o business understanding

In [2]:
# Some boston Image

## Importing libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
# Data understanding
# Vai ser o que está abaixo 

In [5]:
list_df = pd.read_csv("../boston-airbnb-analysis/inputs/listings.csv")

In [6]:
list_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [7]:
print(list_df.shape)

(3585, 95)


In [8]:
null_col = list_df.isnull().sum()[list_df.isnull().sum() > 0].sort_values(ascending=False) / list_df.shape[0]
null_col

jurisdiction_names              1.000000
license                         1.000000
neighbourhood_group_cleansed    1.000000
has_availability                1.000000
square_feet                     0.984379
monthly_price                   0.752301
weekly_price                    0.751185
security_deposit                0.625662
notes                           0.550907
interaction                     0.433473
access                          0.415342
neighborhood_overview           0.394700
host_about                      0.365132
transit                         0.359833
house_rules                     0.332497
cleaning_fee                    0.308787
space                           0.294840
review_scores_accuracy          0.229568
review_scores_location          0.229289
review_scores_value             0.229010
review_scores_checkin           0.228731
review_scores_communication     0.228173
review_scores_cleanliness       0.228173
review_scores_rating            0.226778
first_review    

In [9]:
list_df.isnull().sum(axis=1)[list_df.isnull().sum(axis=1) > 0].sort_values(ascending=False) / list_df.shape[1]

3300    0.368421
1836    0.357895
1923    0.357895
3223    0.347368
1310    0.336842
          ...   
2636    0.052632
2047    0.042105
47      0.042105
446     0.042105
3219    0.042105
Length: 3585, dtype: float64

In [10]:
list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 95 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3585 non-null   int64  
 1   listing_url                       3585 non-null   object 
 2   scrape_id                         3585 non-null   int64  
 3   last_scraped                      3585 non-null   object 
 4   name                              3585 non-null   object 
 5   summary                           3442 non-null   object 
 6   space                             2528 non-null   object 
 7   description                       3585 non-null   object 
 8   experiences_offered               3585 non-null   object 
 9   neighborhood_overview             2170 non-null   object 
 10  notes                             1610 non-null   object 
 11  transit                           2295 non-null   object 
 12  access

In [11]:
# Data Preparation

In [12]:
# Dropping columns with more than 50% null values.
list_df = list_df.drop(null_col.index[:9], axis=1)
print(list_df.shape)

(3585, 86)


In [13]:
# Dropping some columns that are not usefull for my analysis
col_drop =[]
for col in list_df.columns:
    if col[-3:] == "url" or col[-2:] == 'id':
        col_drop.append(col)
col_drop.append('country')
df = list_df.drop(col_drop, axis=1)

In [14]:
def handling_rates(col):
    values = []
    for i in col:
        try:
            values.append(int(i[:-1]) / 100)
        except:
            values.append(i)
    return values

for col in df.columns:
    if 'rate' in col:
        df[col] = handling_rates(df[col])   


In [15]:
# Ajusting price column to float
df['price'] = df.price.apply(lambda p: float(p[1:].replace(',', '')))

In [16]:
def cleaning_fees(col):
    fees = []
    for i in col:
        try:
            fees.append(float(i[1:]))
        except:
            fees.append(i)
    return fees

for col in ['cleaning_fee', 'extra_people']:
    df[col] = cleaning_fees(df[col])
    df[col].fillna(0, inplace = True)

In [19]:
print(df.shape)
df.isna().sum()[df.isna().sum() > 0].sort_values(ascending=False)

(3585, 74)


interaction                    1554
access                         1489
neighborhood_overview          1415
host_about                     1309
transit                        1290
house_rules                    1192
cleaning_fee                   1107
space                          1057
review_scores_accuracy          823
review_scores_location          822
review_scores_value             821
review_scores_checkin           820
review_scores_cleanliness       818
review_scores_communication     818
review_scores_rating            813
reviews_per_month               756
first_review                    756
last_review                     756
neighbourhood                   543
host_response_rate              471
host_response_time              471
host_acceptance_rate            471
host_neighbourhood              339
summary                         143
zipcode                          38
bathrooms                        14
market                           14
host_location               

In [20]:
df.city.unique()

array(['Boston', 'Roslindale', 'Roslindale, Boston', 'Jamaica Plain',
       'Brookline', 'Jamaica Plain, Boston', 'Jamaica Plain (Boston)',
       'Roxbury Crossing', 'Jamaica Plain ', 'Jamaica Plain, MA',
       'Boston (Jamaica Plain)', 'Jamaica plain ', 'ROXBURY CROSSING',
       'Mission Hill, Boston', 'boston', '波士顿',
       'Boston, Massachusetts, US', 'South End, Boston', 'Cambridge',
       'East Boston', 'east Boston ', 'Charlestown',
       'Boston (Charlestown)', 'Somerville', 'Watertown', 'Brighton',
       'Brighton ', 'Boston ', nan, 'West Roxbury', 'Newton', 'Hyde Park',
       'Mattapan', 'Milton', 'Dorchester', 'dorchester, boston ',
       'South Boston', 'Allston', 'ALLSTON'], dtype=object)